# Anti-Aliasing Neural Network Training on Google Colab

This notebook enables GPU-accelerated training of anti-aliasing RNN models.

**Expected speedup:** 20-30x (from 32 min/epoch on CPU to ~1-2 min/epoch on T4 GPU)

## Setup Overview

1. Mount Google Drive for persistent storage
2. Clone repository from GitHub
3. Install dependencies
4. Create symlinks for weights/audio/checkpoints
5. Configure wandb for remote monitoring
6. Run training

## Before Running

Ensure your Google Drive has this structure:
```
Google Drive/
└── AA_Neural/
    ├── weights/
    │   └── NAM/
    │       └── Marshall JCM 800 2203/
    │           └── JCM800 2203 - P5 B5 M5 T5 MV7 G10 - AZG - 700.nam
    ├── audio_data/
    │   └── val_input.wav
    └── checkpoints/
        (created automatically)
```

## Quick Start

**First time setup:**
1. Update `REPO_URL` in cell 2 with your GitHub repository URL
2. Run all cells in order (Runtime → Run all)
3. Authenticate Google Drive when prompted
4. Enter wandb API key when prompted

**Subsequent runs:**
- Just run all cells again
- Previous checkpoints remain in Drive

**Monitor training:**
- Live metrics appear below each cell
- Visit wandb.ai dashboard for full visualization
- Can close this tab - training continues, checkpoints save to Drive

**Troubleshooting:**
- If Drive mount fails: Runtime → Reset all runtimes, try again
- If out of memory: Reduce batch_size in config.py
- If session disconnects: Re-run notebook, checkpoints in Drive are preserved

## 1. Mount Google Drive

This mounts your Google Drive to `/content/drive` for persistent storage.

In [1]:
from google.colab import drive
import os

# Check if Drive is already accessible
if os.path.exists('/content/drive/MyDrive'):
    print("Drive already mounted")
else:
    # Clean up /content/drive if it exists but isn't mounted
    if os.path.exists('/content/drive'):
        import shutil
        shutil.rmtree('/content/drive')
    drive.mount('/content/drive')

# Verify the AA_Neural folder exists
drive_base = '/content/drive/MyDrive/AA_Neural'
assert os.path.exists(drive_base), f"ERROR: {drive_base} not found. Create it and upload weights/audio_data first."
assert os.path.exists(f'{drive_base}/weights'), f"ERROR: {drive_base}/weights not found."
assert os.path.exists(f'{drive_base}/audio_data'), f"ERROR: {drive_base}/audio_data not found."

print("\u2713 Google Drive mounted successfully")
print(f"\u2713 Found {drive_base}")
print(f"\u2713 Found {drive_base}/weights")
print(f"\u2713 Found {drive_base}/audio_data")

Mounted at /content/drive
✓ Google Drive mounted successfully
✓ Found /content/drive/MyDrive/AA_Neural
✓ Found /content/drive/MyDrive/AA_Neural/weights
✓ Found /content/drive/MyDrive/AA_Neural/audio_data


## 2. Clone Repository

Clone the repository from GitHub. Update the URL below with your repo.

In [2]:
import os

# Configuration
REPO_URL = 'https://github.com/7sharp9/dafx25_antialiasing_neural.git'  # UPDATE THIS
REPO_DIR = '/content/dafx25_antialiasing_neural'

# Remove existing clone if present
if os.path.exists(REPO_DIR):
    print(f"Removing existing {REPO_DIR}")
    !rm -rf {REPO_DIR}

# Clone repository
print(f"Cloning {REPO_URL}...")
!git clone --recurse-submodules {REPO_URL} {REPO_DIR}

# Change to repo directory
os.chdir(REPO_DIR)
print(f"\n\u2713 Repository cloned to {REPO_DIR}")
print(f"\u2713 Current directory: {os.getcwd()}")

# Verify OpenAmp submodule
assert os.path.exists('OpenAmp/Open_Amp/amp_model.py'), "ERROR: OpenAmp submodule not loaded correctly"
print("\u2713 OpenAmp submodule loaded")

Cloning https://github.com/7sharp9/dafx25_antialiasing_neural.git...
Cloning into '/content/dafx25_antialiasing_neural'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (276/276), done.
remote: Total 637 (delta 88), reused 315 (delta 58), pack-reused 286 (from 1)
Receiving objects: 100% (637/637), 234.16 MiB | 33.15 MiB/s, done.
Resolving deltas: 100% (95/95), done.
Submodule 'OpenAmp' (https://github.com/Alec-Wright/OpenAmp.git) registered for path 'OpenAmp'
Cloning into '/content/dafx25_antialiasing_neural/OpenAmp'...
remote: Enumerating objects: 847, done.        
remote: Counting objects: 100% (102/102), done.        
remote: Compressing objects: 100% (81/81), done.        
remote: Total 847 (delta 41), reused 73 (delta 20), pack-reused 745 (from 1)        
Receiving objects: 100% (847/847), 269.83 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Submodule path 'OpenAmp': checked out 'fb690191

## 3. Install Dependencies

Install required Python packages. Colab already has PyTorch, so we only install additional dependencies.

In [8]:
# Install core dependencies
!pip install -q pytorch-lightning wandb auraloss neural-amp-modeler librosa torchcodec

# Verify critical imports
import torch
import pytorch_lightning as pl
import wandb

print("\u2713 Dependencies installed")
print(f"\u2713 PyTorch version: {torch.__version__}")
print(f"\u2713 PyTorch Lightning version: {pl.__version__}")
print(f"\u2713 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"\u2713 GPU: {torch.cuda.get_device_name(0)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.4 MB/s eta 0:00:00
✓ Dependencies installed
✓ PyTorch version: 2.9.0+cu126
✓ PyTorch Lightning version: 2.6.0
✓ CUDA available: True
✓ GPU: Tesla T4


## 4. Link Drive Storage

Create symlinks so the training script reads weights/audio from Drive and saves checkpoints to Drive.

In [4]:
# @title
import os
import shutil

drive_base = '/content/drive/MyDrive/AA_Neural'
repo_dir = '/content/dafx25_antialiasing_neural'

# Ensure we're in the repo directory
os.chdir(repo_dir)

# Create symlinks for persistent storage
symlinks = [
    ('weights', f'{drive_base}/weights'),
    ('audio_data', f'{drive_base}/audio_data'),
    ('lightning_logs', f'{drive_base}/checkpoints'),  # Redirect PL checkpoints
]

for link_name, target in symlinks:
    link_path = f'{repo_dir}/{link_name}'

    # Remove existing file/dir/symlink
    if os.path.islink(link_path):
        os.unlink(link_path)
    elif os.path.isdir(link_path):
        shutil.rmtree(link_path)
    elif os.path.exists(link_path):
        os.remove(link_path)

    # Create target directory if it doesn't exist
    os.makedirs(target, exist_ok=True)

    # Create symlink
    os.symlink(target, link_path)
    print(f"\u2713 Linked {link_name} -> {target}")

# Verify critical files exist
test_file = 'weights/NAM/Marshall JCM 800 2203/JCM800 2203 - P5 B5 M5 T5 MV7 G10 - AZG - 700.nam'
assert os.path.exists(test_file), f"ERROR: {test_file} not found. Check that Drive has weights/NAM/Marshall JCM 800 2203/ directory."
print(f"\u2713 Verified model file accessible: {test_file}")

test_audio = 'audio_data/val_input.wav'
assert os.path.exists(test_audio), f"ERROR: {test_audio} not found. Check that Drive has audio_data/val_input.wav file."
print(f"\u2713 Verified audio file accessible: {test_audio}")

print(f"\u2713 Checkpoints will save to {drive_base}/checkpoints")

✓ Linked weights -> /content/drive/MyDrive/AA_Neural/weights
✓ Linked audio_data -> /content/drive/MyDrive/AA_Neural/audio_data
✓ Linked lightning_logs -> /content/drive/MyDrive/AA_Neural/checkpoints
✓ Verified model file accessible: weights/NAM/Marshall JCM 800 2203/JCM800 2203 - P5 B5 M5 T5 MV7 G10 - AZG - 700.nam
✓ Verified audio file accessible: audio_data/val_input.wav
✓ Checkpoints will save to /content/drive/MyDrive/AA_Neural/checkpoints


## 5. Configure wandb

Login to Weights & Biases for experiment tracking. Get your API key from https://wandb.ai/authorize

In [5]:
import wandb

# Login to wandb
# Option 1: Interactive login (will prompt for API key)
wandb.login()

# Option 2: Programmatic login (uncomment and add your key)
# wandb.login(key='your-api-key-here')

print("\u2713 wandb configured")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 7sharp9 (7sharp9-moirae-software-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✓ wandb configured


## 6. Training Configuration

Set training parameters. Modify these as needed.

In [6]:
# Training configuration
CONFIG_IDX = 3          # Model config (3 = JCM800 NAM model)
MAX_EPOCHS = 100        # Number of epochs to train
USE_WANDB = True        # Enable wandb logging
LOG_EVERY_N_STEPS = 10  # Log every 10 steps (adjust as needed)

print(f"Training configuration:")
print(f"  Config index: {CONFIG_IDX}")
print(f"  Max epochs: {MAX_EPOCHS}")
print(f"  wandb logging: {USE_WANDB}")
print(f"  Log every N steps: {LOG_EVERY_N_STEPS}")

Training configuration:
  Config index: 3
  Max epochs: 100
  wandb logging: True


## 7. Run Training

Execute the training script. Progress will stream to this notebook.

**Expected time:** ~2-3 hours for 100 epochs on T4 GPU

**Monitoring:**
- Live metrics visible in this notebook
- Full dashboard at wandb.ai
- Checkpoints auto-save to Drive

In [ ]:
import os

# Ensure we're in the repo directory
os.chdir('/content/dafx25_antialiasing_neural')

# Build command
cmd_parts = [
    'python train.py',
    f'--config {CONFIG_IDX}',
    f'--max_epochs {MAX_EPOCHS}',
]

if USE_WANDB:
    cmd_parts.append('--wandb')
else:
    cmd_parts.append('--no-wandb')

cmd = ' '.join(cmd_parts)

print(f"Executing: {cmd}")
print("="*80)

# Run training (this will take a while)
!{cmd}

Executing: python train.py --config 3 --max_epochs 100 --wandb
Configuration: JCM800
Model: weights/NAM/Marshall JCM 800 2203/JCM800 2203 - P5 B5 M5 T5 MV7 G10 - AZG - 700.nam
CPU cores: 2
Seed set to 42
AmpModel(
  (model): WaveNet(
    (_net): _WaveNet(
      (_layers): ModuleList(
        (0): _Layers(
          (_rechannel): Conv1d(1, 16, kernel_size=(1,), stride=(1,), bias=False)
          (_layers): ModuleList(
            (0): _Layer(
              (_conv): Conv1d(16, 16, kernel_size=(3,), stride=(1,))
              (_input_mixer): Conv1d(1, 16, kernel_size=(1,), stride=(1,), bias=False)
              (_activation): Tanh()
              (_1x1): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
            )
            (1): _Layer(
              (_conv): Conv1d(16, 16, kernel_size=(3,), stride=(1,), dilation=(2,))
              (_input_mixer): Conv1d(1, 16, kernel_size=(1,), stride=(1,), bias=False)
              (_activation): Tanh()
              (_1x1): Conv1d(16, 16, kernel_size

## 8. Post-Training

After training completes, checkpoints are in Google Drive.

In [ ]:
# List saved checkpoints
import os
import glob

checkpoints_base = '/content/drive/MyDrive/AA_Neural/checkpoints'

# Find all checkpoint versions
versions = sorted(glob.glob(f'{checkpoints_base}/version_*'))

if versions:
    latest_version = versions[-1]
    print(f"Found {len(versions)} training run(s)")
    print(f"\nLatest run: {latest_version}")

    # List checkpoints in latest run
    checkpoints = glob.glob(f'{latest_version}/checkpoints/*.ckpt')
    if checkpoints:
        print(f"\nCheckpoints ({len(checkpoints)}):")
        for ckpt in sorted(checkpoints):
            size_mb = os.path.getsize(ckpt) / (1024*1024)
            print(f"  {os.path.basename(ckpt)} ({size_mb:.1f} MB)")

    # Check for best model export
    best_export = f'{latest_version}/best_export'
    if os.path.exists(best_export):
        print(f"\n✓ Best model exported to: {best_export}")
        exported_files = glob.glob(f'{best_export}/*')
        for f in exported_files:
            print(f"  {os.path.basename(f)}")
else:
    print("No checkpoints found. Training may not have completed.")

print(f"\nAll results saved to Google Drive: {checkpoints_base}")